In [1]:
import os
import numpy as np
import tensorflow as tf
from vgg16 import *
from classes import class_names

In [2]:
# Misc
LOG_DIR = './log/'

# Training Parameters
LEARNING_RATE = 0.005
MOMENTUM = 0.9
NUM_EPOCHS = 1
NUM_STEPS = 500
BATCH_SIZE = 128
DISPLAY_STEP = 10

# Network Parameters
NUM_CLASSES = 1000
KEEP_RATE = 0.75

In [3]:
def read_image (path):
    # Read Image
    img_file = tf.read_file(path)
    img_decoded = tf.image.decode_jpeg(img_file, channels=3)
    img_resized = tf.image.resize_images(img_decoded, [227, 227])
    
    # Convert to BGR <-- Because the weights were trained from opencv images
    img = tf.reverse(img_resized, axis=[-1])
    
    # Subtract from mean
    imagenet_mean = np.array([104., 117., 124.])
    img -= imagenet_mean
    
    return img
    

In [4]:
model_graph = tf.Graph()
with model_graph.as_default():
    
    data = tf.data.Dataset.from_tensor_slices(np.array(['../test_image/zebra.jpeg']))
    data = data.map(read_image).batch(1)
    iterator = data.make_one_shot_iterator()
    next_el = iterator.get_next()
    
    #images = tf.placeholder(tf.float32, [None, 227, 227, 3])
    #labels = tf.placeholder(tf.float32, [None, NUM_CLASSES])
    keep_prob = tf.placeholder(tf.float32)
    
    # Build Model
    vgg = MY_VGG16(x=next_el, keep_rate=keep_prob, 
                         num_classes=NUM_CLASSES, skip_layers=[],
                         weights_path='../../vgg16.npy')
    vgg.build()
    
    # Logits and Predictions
    logits = vgg.logits
    prediction = {'classes': tf.argmax(logits, axis=1), 
                  'prob': tf.nn.softmax(logits, name='prob')}
    
    init = tf.global_variables_initializer()

In [6]:
with tf.Session(graph=model_graph) as sess:
    
    sess.run(init)
    
    vgg.load_weights(session=sess, encoding='latin1')
    
    pred = sess.run(prediction, feed_dict={keep_prob: 1.0})
    
    print(class_names[pred['classes'][0]])
    print(pred['prob'].max())

InvalidArgumentError: Input to reshape is a tensor with 32768 values, but the requested shape requires a multiple of 25088
	 [[Node: flatten6 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](pool5, flatten6/shape)]]

Caused by op 'flatten6', defined at:
  File "/Users/ranranking/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/ranranking/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/ranranking/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/ranranking/miniconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/ranranking/miniconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/Users/ranranking/miniconda3/lib/python3.

In [5]:
writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'model_1'), graph=model_graph)

In [4]:
weights = np.load('../../vgg16.npy', encoding='latin1').item()

In [8]:
for i in weights:
    print(i)

conv5_1
fc6
conv5_3
conv5_2
fc8
fc7
conv4_1
conv4_2
conv4_3
conv3_3
conv3_2
conv3_1
conv1_1
conv1_2
conv2_2
conv2_1
